In the CLIP paper we dont take the i_k dim vs the t_k+x dimension because each dimension be it in text modality or image modality it should take up the same meaning , a dimension 'k' should represent the same in both the vectors as they are coming from the same latent space !!

In [1]:
import torch
import torch.nn as nn 
import os 
import matplotlib.pyplot as plt

FILE_PATH = os.getcwd()
print(FILE_PATH)

/Users/mohitdulani/Desktop/personal/sencillo-robotics


In [2]:
# This is a ViT model 
from datasets import load_dataset
from PIL import Image  # force Pillow import


# lets take a small image 
# ds = load_dataset("itsmohit/stl10_5percent", split='test',  cache_dir=FILE_PATH, )
ds = load_dataset('mnist', split='train[:100]', cache_dir = FILE_PATH)
print(ds)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 100
})


![CLIP Model](assets/images/clip.png)

--- 

CLIP ( Contrastive Language Image Pretraining  )

1. Image - text retrieval ( finding the caption (not generating it) ) 
2. Zero Shot Classification, acts as a classifier ( A Image of dog, Image of cat .. )
3. Feature Extraction ( its anyways a ViT backbone on trained text space as well )


--- 


**Vision Model** : So the vision encoder is same as the ViT backbone it takes in the image , makes patches then converts them to tokens that's the encoder



In [ ]:
# Here we need an image encoder to extract the features from the images (Can use resnet as well as ViT) also we need and text encoder (transformers rules best) and then use it to joinly train both ! 


from transformers import CLIPConfig, CLIPModel


MODEL_NAME = "openai/clip-vit-base-patch16"
# 1️⃣ Load the config only (does not download weights)
config = CLIPConfig.from_pretrained(MODEL_NAME)

# 2️⃣ Initialize the model with random weights
model = CLIPModel(config)

print(model)



config.json: 0.00B [00:00, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [ ]:
## Vision encoder

class ImageEncoder(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.vision_encoder = model.vision_model 
        self.vision_projection = model.visual_projection

    def forward(self, x):
        # x -> image tensor (ch , H,  W)

        x = self.vision_encoder(x)
        x = self.vision_projection(x) 
        return x 


# text encoder


class TextEncoder(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.text_encoder = model.text_model
        self.text_projection = model.text_projection

    def forward(self, x):

        x = self.text_encoder(x)
        x = self.text_projection(x)

        return x 

In [ ]:
import inspect

print(inspect.getsource(model.forward))
# The actual code for how the forward method works in these models !! 


### Loss function

---

Image_embedding : BackBone Image extractor model i.e. [n , d_e]  
Text_embedding  : BackBone Text extractor model i.e. [n , d_e]

--- 

logits_before = np.dot(I_e, T_e.T) 
logits = logits_before * np.exp(t) ## why ? 
